Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-02 21:21:14--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.121.66
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.121.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  52.9MB/s    in 3.7s    

2019-10-02 21:21:18 (52.9 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
import pandas as pd

In [0]:
items = ['Banana', 'Bag of Organic Bananas', 'Organic Strawberries',
         'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado',
         'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk']

In [0]:
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [8]:
order_products__prior.head(2) #needed, columns = ['order_id', 'product_id']

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1


In [9]:
order_products__train.head(2) #needed, columns = ['order_id', 'product_id']

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1


In [10]:
orders.head(2) #not needed

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [11]:
products.head(2) #needed, columns  =['product_id', 'product_name']

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13


In [0]:
##### YOUR CODE HERE #####

#Clean needed dataframes of unnessecary columns
order_products__prior_clean = order_products__prior[['order_id', 'product_id']]
order_products__train_clean = order_products__train[['order_id', 'product_id']]
products_clean = products[['product_id', 'product_name']]

In [13]:
#Concatenating the like dataframes (order_products__prior_clean and order_products__train_clean)
order_products_clean = pd.concat([order_products__train_clean,order_products__prior_clean])
order_products_clean.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [14]:
#Merging order_products_clean with products_clean to get all necessary data into one dataframe
merged = pd.merge(order_products_clean, products_clean, how='inner', on='product_id')
merged.head(2)

,order_id,product_id,product_name
0,1,49302,Bulgarian Yogurt
1,816049,49302,Bulgarian Yogurt


In [15]:
#Count number of orders each item was in (How many times each item was ordered)
merged['product_name'].loc[merged['product_name'].isin(items)].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
import seaborn as sns

In [17]:
flights = sns.load_dataset('flights')
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [18]:
##### YOUR CODE HERE #####
flights.pivot_table(index='year', columns='month', values='passengers')

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [29]:
orders.head(2)

,order_id,order_hour_of_day
0,2539329,8
1,2398795,7


In [57]:
##### YOUR CODE HERE #####
#Clean needed dataframes (and/or use those that were used previously)
orders_clean = orders[['order_id', 'order_hour_of_day']]
orders_clean['order_hour_of_day'] = orders['order_hour_of_day'].loc[orders['order_hour_of_day'].isin(range(19,24))]
orders_clean.dropna(inplace=True)
orders_clean.head(2)

,order_id,order_hour_of_day
27,444309,19.0
55,2402008,19.0


In [0]:
l = merged['product_name'].value_counts()
sorted(l)
l.drop
merged_clean = merged.drop(labels=l)

In [63]:
merged_clean.head(2)

,order_id,product_id,product_name
0,1,49302,Bulgarian Yogurt
867,432356,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese


In [64]:
#Merge all the relevent Dataframes for the required analysis
merged_0 = pd.merge(merged_clean, orders_clean, how='inner', on='order_id')
print(merged_0.shape)
merged_0.head(2)

(4247799, 4)


,order_id,product_id,product_name,order_hour_of_day
0,634893,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,19.0
1,634893,23986,Grassfed Whole Milk Plain Yogurt,19.0


In [65]:
merged_0['product_name'].value_counts()[:25]

Banana                      61711
Bag of Organic Bananas      50600
Organic Strawberries        37462
Organic Baby Spinach        32727
Organic Hass Avocado        28343
Organic Avocado             23200
Organic Whole Milk          19807
Organic Raspberries         19079
Strawberries                18998
Large Lemon                 18820
Limes                       17474
Organic Blueberries         14747
Organic Zucchini            14275
Organic Yellow Onion        13913
Organic Garlic              13668
Cucumber Kirby              13066
Organic Grape Tomatoes      11826
Organic Lemon               11489
Seedless Red Grapes         11401
Organic Cucumber            11105
Organic Fuji Apple          11020
Apple Honeycrisp Organic    10776
Organic Baby Carrots        10616
Honeycrisp Apple            10422
Carrots                     10190
Name: product_name, dtype: int64

In [66]:
merged_0.sort_values(by=['order_hour_of_day'],ascending=False)

,order_id,product_id,product_name,order_hour_of_day
4247798,2280826,35941,LittleSnug sz,23.0
496723,1144405,13176,Bag of Organic Bananas,23.0
496717,1144317,28204,Organic Fuji Apple,23.0
496718,1144317,31883,Vegetable Lasagna,23.0
496719,1144317,47630,Medium Navel Orange,23.0
496720,1144317,25472,Sparkling Tangerine Probiotic Drink,23.0
496721,1144317,45437,Sea Salt Made With Organic Grain Rice Chips,23.0
496722,1144317,33315,Organic Not Salted Pistachios,23.0
496724,1144405,45007,Organic Zucchini,23.0
496753,1144546,35951,Organic Unsweetened Almond Milk,23.0


## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####